In [1]:
import redis
from pymongo import MongoClient
from bson import ObjectId
import json
#install redis-> pip install redis
#pip install pymongo

def obtenerAnalisis():
    redis_client=redis.Redis(host='localhost', port=6379, decode_responses=True)
    
    #/// Primero verrificar antes de la consulta a la BD si redis tiene la info
    cached_analisis=redis_client.lrange('admin_2', 0, -1) #busca los objetos como key type list #usar id del admin y id del doc de mongo
    if cached_analisis: 
        print("entre cache")
        return json.loads(cached_analisis[0])
    
    mongo_client = MongoClient('localhost', 27017, username='root', password='example')
    db = mongo_client.Resultados
    collection = db.analisis
    resultados = collection.find_one({"_id": ObjectId('6616b7c377fa1afa40052503')})
    resultados['_id'] = str(resultados['_id'])
    
    redis_client.lpush("admin_2",json.dumps(resultados)) # se le setea la info a redis
    redis_client.expire("admin_2", 100) #asigna 4000s para que expire

    redis_client.close()

    return resultados

print(obtenerAnalisis())

{'_id': '6616b7c377fa1afa40052503', 'cantidadEncuestados': 27316, 'respuestasA_uno': 26, 'respuestasB_uno': 200, 'respuestasC_uno': 1000, 'repuestasD_uno': 26090}
